### Train optimizer
Notebook to train different optimizer to compare

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import pickle
import os
import sys
import time
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import trange, tqdm

In [13]:
# Run this cell only if you run this notebook on Google Colab.
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/Colab_Notebooks/OptiML/OptiForML2022')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from optimizer.AdaHessian import AdaHessian
from optimizer.Atmo import Atmo, MASScheduler
from optimizer.AdaSGD import AdaSGD, AdaSGDscheduler
from dataset import ImagesDataset
from model import ResNet18
from path import TRAIN_HISTORY_DIR, TRAIN_MODEL_DIR

In [15]:
# choose dataset
#data_name = "MNIST"
data_name = "cifar"
full = True
tiny = False

# choose optimizer
optimizer_name = "adam"
optimizer_name = "sgd"
optimizer_name = "atmo"
optimizer_name = "dynamicAtmo"
#optimizer_name = "adaHessian"
#optimizer_name = "adaSGD"

# choose scheduler
#scheduler_name = None
scheduler_name = "cosineAnnealinglr"
#scheduler_name = "steplr"
#scheduler_name = "multiSteplr"

# choose nb of epochs
epochs = 100
#if tiny:
#  epochs = 50
#else:
#  epochs = 10

In [16]:
# load dataset
train_dataset = ImagesDataset(full = full, tiny=tiny, cifar=(data_name=="cifar"))
test_dataset = ImagesDataset(full = full, tiny=tiny, cifar=(data_name=="cifar"), test=True)


trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=True)

* Using CIFAR
Files already downloaded and verified
** Use 50000 train samples
* Using CIFAR
Files already downloaded and verified
** Use 10000 test samples


In [17]:
# load model
model = ResNet18(in_channel=1 if data_name=="MNIST" else 3)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)
model.train()
model.to(device)
#model.load_state_dict(torch.load(os.path.join(TRAIN_MODEL_DIR, f"{optimizer_name}_{scheduler_name}_{epochs}epochs_{data_name}.pt")))

dynamic = None

# load optimizer
if optimizer_name == "adam":
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
if optimizer_name == "sgd":
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
if optimizer_name == "atmo":
    optimizer = Atmo(model.parameters())
if optimizer_name == "dynamicAtmo":
    optimizer = Atmo(model.parameters(), lr = 0.01, adam_w=1, sgd_w=0, momentum = 0.95, weight_decay = 0.0005)
    dynamic = MASScheduler(optimizer, epochs = epochs)
if optimizer_name == "adaHessian":
    optimizer = AdaHessian(model.parameters(), lr = 0.15)
if optimizer_name == "adaSGD":
    optimizer = AdaSGD(model.parameters(), lr = 0.1, ada_w = 1, sgd_w = 0)
    dynamic = AdaSGDscheduler(optimizer, epochs = 100) # do only 100 epochs with both optim then only sgd

# load scheduler
if scheduler_name is not None:
    if scheduler_name == "cosineAnnealinglr":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, verbose=True)
    if scheduler_name == "steplr":
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, verbose=True)
    if scheduler_name == "multiSteplr":
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [80, 120], gamma = 0.1, verbose = False)



criterion = nn.CrossEntropyLoss()
#with open(os.path.join(TRAIN_HISTORY_DIR, f"log-{optimizer_name}_{scheduler_name}_{epochs}epochs_{data_name}.pickle"), 'rb') as f:
#    history = pickle.load(f)

cuda
Adjusting learning rate of group 0 to 1.0000e-02.


In [18]:
train_loss = [] #history['train_loss']
train_acc = [] #history['train_acc']
test_loss = [] #history['test_loss']
test_acc = [] #history['test_acc']
time_epoch = [] #history['time_epoch']
epochs_done = 1 #len(train_loss)

with trange(epochs_done, epochs + 1, desc='Training', unit='epoch') as t:
  for epoch in t:
    losses = []
    acc = []
    start_time = time.time()
    with tqdm(trainDataLoader, desc=f'Train epoch {epoch}',
              unit='batch', leave=False) as t1:
      for x_train, y_train in t1:
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward(create_graph=(optimizer_name=="adaHessian"))
        optimizer.step()

        losses.append(loss)
        pred = torch.argmax(output, axis = 1)
        acc.append(sum(pred == y_train).item()/pred.shape[0])

    if dynamic is not None:
          dynamic.step()

    train_loss.append(sum(losses)/len(losses))
    train_acc.append(sum(acc)/len(acc))

    losses = []
    acc = []
    with torch.no_grad():
      with tqdm(testDataLoader, desc=f'Test epoch {epoch}',
                unit='batch', leave=False) as t1:
        for x_test, y_test in t1:
          x_test = x_test.to(device)
          y_test = y_test.to(device)

          output = model(x_test)
          loss = criterion(output, y_test)
          losses.append(loss)

          pred = torch.argmax(output, axis = 1)
          acc.append(sum(pred == y_test).item()/pred.shape[0])

      test_loss.append(sum(losses)/len(losses))
      test_acc.append(sum(acc)/len(acc))

      end_time = time.time()
      time_epoch.append(end_time-start_time)
    if scheduler_name is not None:
          scheduler.step()
    history = dict()
    history["train_loss"] = train_loss
    history["train_acc"] = train_acc
    history["test_loss"] = test_loss
    history["test_acc"] = test_acc
    history["time_epoch"] = time_epoch
    history["data"] = data_name
    history["model"] = "resnet18"
    history["epochs"] = epochs
    history["optimizer"] = optimizer_name
    with open(os.path.join(TRAIN_HISTORY_DIR, f"log-{optimizer_name}_{scheduler_name}_{epochs}epochs_{data_name}_lr_0.01.pickle"), 'wb') as f:
      pickle.dump(history, f)
    weights = model.state_dict()
    torch.save(weights,os.path.join(TRAIN_MODEL_DIR, f"{optimizer_name}_{scheduler_name}_{epochs}epochs_{data_name}_lr_0.01.pt"))
#mean by epoch
time_epoch = sum(time_epoch)/len(time_epoch)

Training:   0%|          | 0/100 [00:00<?, ?epoch/s]

Train epoch 1:   0%|          | 0/49 [00:00<?, ?batch/s]

Test epoch 1:   0%|          | 0/10 [00:00<?, ?batch/s]

Adjusting learning rate of group 0 to 9.9975e-03.


Train epoch 2:   0%|          | 0/49 [00:00<?, ?batch/s]

...

Test epoch 99:   0%|          | 0/10 [00:00<?, ?batch/s]

Adjusting learning rate of group 0 to 2.4672e-06.


Train epoch 100:   0%|          | 0/49 [00:00<?, ?batch/s]

Test epoch 100:   0%|          | 0/10 [00:00<?, ?batch/s]

Adjusting learning rate of group 0 to 0.0000e+00.


In [19]:
# run this cell to save history
history = dict()
history["train_loss"] = train_loss
history["train_acc"] = train_acc
history["test_loss"] = test_loss
history["test_acc"] = test_acc
history["time_epoch"] = time_epoch
history["data"] = data_name
history["model"] = "resnet18"
history["epochs"] = epochs
history["optimizer"] = optimizer_name
with open(os.path.join(TRAIN_HISTORY_DIR, f"log-{optimizer_name}_{epochs}epochs_{data_name}_lr_0.01.pickle"), 'wb') as f:
  pickle.dump(history, f)

In [20]:
# run this cell to save model
weights = model.state_dict()
torch.save(weights,os.path.join(TRAIN_MODEL_DIR, f"{optimizer_name}_{epochs}epochs_{data_name}_lr_0.01.pt"))